In [2]:
import requests
import json
from os import environ


Retrieving Project - testing API key

In [8]:
project_name = "Traffic Sign Detection"
url = "https://api.scale.com/v1/"

headers = {
    "Accept": "application/json",
    "Authorization": "Basic "+environ.get('Bearer')
}

response = requests.request("GET", url+"projects/"+project_name, headers=headers)
pretty_project_name = json.loads(response.text)
print (json.dumps(pretty_project_name, indent=4))


{
    "type": "annotation",
    "pipelineName": "standard_task",
    "created_at": "2020-07-18T01:20:22.280Z",
    "param_history": [
        {
            "created_at": "2020-12-17T23:18:07.540Z",
            "version": 0,
            "patch": false
        }
    ],
    "name": "Traffic Sign Detection",
    "pinned": true,
    "archived": false,
    "customGradersUsedOrDismissed": false,
    "edgeCaseAlertsEnabled": true,
    "turnOnLongHints": true,
    "isAudioTranscription": false,
    "useLabelerManagementPlatform": false,
    "containsAdultContent": false,
    "useTextCollectionTemplate": false,
    "datasetLinks": [],
    "taxonomy": {
        "patch": false
    }
}


Retrieving Tasks

In [ ]:

response = requests.request("GET", url+"tasks?status=completed&project="+project_name, headers=headers)

task_list_json = json.loads(response.text)

# list of tasks
tasks = task_list_json["docs"] 

for i in tasks:
    # print (json.dumps(i["task_id"], indent=4))
    response_task = requests.request("GET", url+"task/"+i["task_id"], headers=headers)
    response_task_json = json.loads(response_task.text)
    print (json.dumps(response_task_json, indent=4))

# Phase 2: one QC

In [14]:
# verify if the bigger annotations have sufficient info within them.
def check_box_size(i):
    annotations=i['response']['annotations']
    # annotations is a list of different labels. 
    for j in annotations:
        annotation_flag = False
        # print(j['label'],annotations.index(j))
        if j['width']*j['height'] > 0.2*1396*850:
            if j['label'] == 'non_visible_face':
                if j['attributes']['background_color'] == 'not_applicable':
                    annotation_flag = True
                    print('ERROR: Big box containing no information in {} tag of task {} '.format(j['uuid'],i['task_id'] ))
        
    if annotation_flag == False:  
        print ('Task {} passed big box test '.format(i['task_id']))

In [15]:
response = requests.request("GET", url+"tasks?status=completed&project="+project_name, headers=headers)

task_list_json = json.loads(response.text)

# list of tasks
tasks = task_list_json["docs"] 

for i in tasks:
    # print (json.dumps(i["task_id"], indent=4))
    response_task = requests.request("GET", url+"task/"+i["task_id"], headers=headers)
    response_task_json = json.loads(response_task.text)
    check_box_size(i)
    # print (json.dumps(response_task_json, indent=4))

Task 5f127f6f26831d0010e985e5 passed big box test 
Task 5f127f6c3a6b1000172320ad passed big box test 
Task 5f127f699740b80017f9b170 passed big box test 
Task 5f127f671ab28b001762c204 passed big box test 
Task 5f127f643a6b1000172320a5 passed big box test 
ERROR: Big box containing no information in 25e6e90c-7acb-4786-a0fb-ca7b10fc3687 tag of task 5f127f5f3a6b100017232099 
Task 5f127f5ab1cb1300109e4ffc passed big box test 
Task 5f127f55fdc4150010e37244 passed big box test 
